In [1]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [2]:
import os
from Bio import SeqIO
import subprocess

family_group = "9.Papilionidae"

output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Vanessa_tameamea"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            error_exons += coordinate_file_list[i]
        
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "26":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Battus_philenor
GCA_028537555.1_ilBatPhil1.0.hap1_genomic.fna
17013501 17013589
Getting Gene


/home/saurav/.local/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


MDNMDDSENNAKISDSAYSNSCSNSQSRR
>Battus_philenor_Exon_1_JAQMYT010000001.1_17013501_17013589_left_0_right_2
ATGGACAATATGGACGACTCAGAGAACAACGCCAAGATATCGGATTCAGCTTACTCCAACAGCTGCAGCAACAGCCAGTCTAGAAGAAG
17013868 17013945
Getting Gene
>Battus_philenor_Exon_2_JAQMYT010000001.1_17013868_17013945_left_1_right_2
tTCTAAATCGACGCATTCTGGCAGCAACTCGAGTGGGAGCAGCGGTTATGGCGGGAAACCTTCCACATCCAGCTACAg
17016936 17017099
Getting Gene
>Battus_philenor_Exon_3_JAQMYT010000001.1_17016936_17017099_left_1_right_1
taaCAACCTCACCCAACCGCCCGAGAATAGAATAAAAGAGAAGGATTCCAAGAAGAAAAAGTCACAGACAGAGAAGTTGACTACAGAGGCGAGGAATGAAAATGAGGGCGAACTGCCCGGACCAGCGCCGATCTTTGAAACACCGAAAGAAGTCACCGACGATG
17018286 17018386
Getting Gene
>Battus_philenor_Exon_4_JAQMYT010000001.1_17018286_17018386_left_2_right_0
GTGACCTGCCATCGCCCGCGCTCGATTCCTCCGAGAAGGGACCGGAGAGTATGGATATCGCAACACCCGATAGCGGTAGTACCAAGGAGGAAATAGCAATA
17018965 17019039
Getting Gene
>Battus_philenor_Exon_5_JAQMYT010000001.1_17018965_17019039_left_0_right_0
tttggaCCATGTGCGACATCACCGGCCAATGTATC

>Ornithoptera_alexandrae_Exon_11_JARDVY010000114.1_946083_946202_left_0_right_0
GGTCCCGCCAATCCCGACGTATTCCAGTCGCCCGAGCCGAATAAAATTTGGAAACTCTCCGAGGAGGAGAAACTGAAAGCTCAATCCATTCGGGACAGTATtcgcaaaattttaaatgag
946987 947136
Getting Gene
>Ornithoptera_alexandrae_Exon_12_JARDVY010000114.1_946987_947136_left_0_right_0
GCCCTTACTAAGCCCGCTGAGGTGGCTAAACAGCAGATGAACAAAAGATGTCAAGACCTGGCCACGTTTATGGAGAGCCTCCTCGAGGACAAGCCAAAGATCGATGAGGAGATGCGCCTGGAGATTCAGGACCCAGAGCACAGCTACTGT
947329 947447
Getting Gene
>Ornithoptera_alexandrae_Exon_13_JARDVY010000114.1_947329_947447_left_0_right_2
GAACGGGACTCGGTGATGCTCGGTGGCATTTCACCCCACCACGACCATAACGACAGCAAATCCGGTACAGAGACGCCGATCAGTTACAACCAGCTAAACTACAACGAGACGCTGCAGAG
947988 948077
Getting Gene
>Ornithoptera_alexandrae_Exon_14_JARDVY010000114.1_947988_948077_left_1_right_2
atatttCGAAAGCCACTTACCATATGAGGACTGCCACACAGTAACCAGTGAAAATGTCTTGGCGACGGATGAACAAATATCCAACGCGAA
948414 948545
Getting Gene
>Ornithoptera_alexandrae_Exon_15_JARDVY010000114.1_948414_948545_left_1_right_2
GTTTCGCTT

821091 821282
Getting Gene
>Ornithoptera_priamus_Exon_21_JARDWD010000410.1_821091_821282_left_0_right_0
AGCACTCACACCGTCCACGATCAGAACAGAAAGACTAAGGCGATGAGGTTGGGCAACAGTGACGACACAATAGACAGGACAGATGGGGAGTCTAGCTATTCGTCGTTTTATTCGTCGTTCTTCAAGACCGAGTCTGGCAGCGCTGAAGAGAGCAGTGACACGAAGAAGCAGACGAGAAAAGGCGAGAACAAG
820607 820699
Getting Gene
>Ornithoptera_priamus_Exon_22_JARDWD010000410.1_820607_820699_left_0_right_0
AACCGGTTAGATGCTGGCTTCTCGCGCGGCGTCAAAGCACAGGAGACGAAACAAGTGCCGCGCAGGAGAATGGATCCGCCCTGGATGAACCAG
820362 820463
Getting Gene
>Ornithoptera_priamus_Exon_23_JARDWD010000410.1_820362_820463_left_0_right_0
GTGTGCCTTACGTCGGATTTGATATATAAGTACCAAGTGTTAACCAAGAGCATGGAGGAGGTGCTGGCGTGCGACCTGCAGAAAATGAAGACGCTGGAGCAG
820090 820236
Getting Gene
>Ornithoptera_priamus_Exon_24_JARDWD010000410.1_820090_820236_left_0_right_0
ccTGCATTGGTGAACGAACAACTCGGTCAACTGTACCTGGATCTACAGCTGGAAGGCGTAGCCGCCAGGTTGACGCTCGAGGAAGGCATAACCAGCTCGAGCAGCTCGGGGGAGGATAACTATGATAGAGCTACTGGAAAGAACTCG
819109 819210
Getting Gene
>Ornithoptera_priam

...

>Papilio_polytes_Exon_25_NW_013528315.1_504279_504380_left_0_right_0
TCACAAAAGAAGAAACGCGAGTACAGCAAACTGGTGATGATATTCGAAGAGAACGCACCCCTCCCTCCCCCGGATTCTCCCCCAGCAGCTTCAGTGTCGTAA
Papilio_protenor
GCA_029286645.1_kiz_Ppr_v1_genomic.fna
3547355 3547443
Getting Gene
MDSMDDSENNAKISDSAYSNSCSNSQSRR
>Papilio_protenor_Exon_1_JAGSMZ010000042.1_3547355_3547443_left_0_right_2
ATGGATAGCATGGACGACTCAGAGAATAATGCGAAGATATCTGATTCAGCTTACTCTAATAGTTGCAGTAACAGTCAGTCCAGGCGCag
3548298 3548375
Getting Gene
>Papilio_protenor_Exon_2_JAGSMZ010000042.1_3548298_3548375_left_1_right_2
TTCGAAGTCAACCCACTCTGGAAGCAATTCGAGTGGCAGTAGCGGATATGGCGGGAAACCATCTACATCCGGTTACAG
3549112 3549278
Getting Gene
>Papilio_protenor_Exon_3_JAGSMZ010000042.1_3549112_3549278_left_1_right_1
taGCAACAACATTAATCAACCACAAGAGAAAAGGCTGAAAGAGAAAGATTCTAAAAAGAAGAAGCAAGCGCATACGGAGATGTTAACTGAGGCGCGAAATGAAAGTGATGATGGTCTTCCAGAATCGGCGCCTATTTTTGAAACCCCCAAAGAAGATATTGACATTG
3551359 3551471
Getting Gene
>Papilio_protenor_Exon_4_JAGSMZ010000042.1_3551359_3551471_left_2_r

>Papilio_xuthus_Exon_10_NW_013530472.1_3089875_3089986_left_1_right_0
GATGAGAGCACAAAATGGCGATTTCGTAAGACTAGAGACAGAATGGTCTTCATTTATCAATCCCTGGTCGAAAAAACTTGAGTTCGTTATTGGAAAACACTATATCATTGAA
3090388 3090507
Getting Gene
>Papilio_xuthus_Exon_11_NW_013530472.1_3090388_3090507_left_0_right_0
GGACCTTTAAATCCTGATGTGTTTCAATCGCCTGATCCCAACAAAGTTCAAAAATTTTCTGAAGAAGACAAGTTAAAAGCTCAATCTCTGAGAGAgagcattattaaaattttaaatgaa
3090815 3090964
Getting Gene
>Papilio_xuthus_Exon_12_NW_013530472.1_3090815_3090964_left_0_right_0
GTTCTGACTAAGCCAGCTGAAGTTGCAAAGCAACAGATGAGCAAGAGATGCCAAGACATAGCTAAATTTATGGAAAGCCTTTTAGAAGATCAACCAAagaatgatgatgatattcgTATAGAACCGCAAGATCCTGATCACAGCTACtac
3091450 3091568
Getting Gene
>Papilio_xuthus_Exon_13_NW_013530472.1_3091450_3091568_left_0_right_2
GAACGTGACTCAGTGATGCTGGGAGGTATATCGCCGCACCATGACTATAACGATAGCAAGTCAAGCACTGAGACTCCAGTCAGCTACAATCAGCTTAATTACAACGACACATTGCAACG
3092026 3092121
Getting Gene
>Papilio_xuthus_Exon_14_NW_013530472.1_3092026_3092121_left_1_right_2
gtATTTCGAGAGTCATCCATCTTGTTCGT

>Parnassius_glacialis_Exon_20_JAVANC010000004.1_34921353_34921516_left_2_right_0
TGCAATGTATGATGTACGGACAAGCAGTATACGGATCGCCAATTATGTATTCACCAATCAATCCCCAAGTCTCTTATCCAGTACAACAAAACTATATGACGCATGACGTTCAGTATATAAACAACATAAACTCACTAGGCTTAGCGAACAGTAACTATGTAGAG
34933198 34933395
Getting Gene
>Parnassius_glacialis_Exon_21_JAVANC010000004.1_34933198_34933395_left_0_right_0
AGCTCAGAGGCTATACGCGATATAATACAGACTAACAATGCGAAATCAATGCGGTGGAGCAACAGCGAGGAGACGATAGAAAAAACTGATGGAGAGTCTAGTTATTCATCATTAAGTTCATCGTTCTTCAAAACTGAATCGGGCAGTGGGGCACAAAGCAGTGACTCTAAAAACGATTTCGGAAAAGAAGGAAAAAAG
34935656 34935754
Getting Gene
>Parnassius_glacialis_Exon_22_JAVANC010000004.1_34935656_34935754_left_0_right_0
AGTGCATTCGATAGCGGCCACGGATACTGGTCAGCATTTGAAACGGAGCAGGTGAAAAACAGTGTTATAAGAAAAATGGAACCACCATGGATGGAACAC
34936870 34936971
Getting Gene
>Parnassius_glacialis_Exon_23_JAVANC010000004.1_34936870_34936971_left_0_right_0
GTTTGTGTTACTTCTGAATTGATCTATAAATATCAGATTCTTACAAAGAGCATGGATGAAGTACTGTCGGGTgataaacagaaaataaaaaccCTTGAACAG
34938349 3493848

1720222 1720353
Getting Gene
>Troides_aeacus_Exon_6_CM067268.1_1720222_1720353_left_0_right_0
GACGGTTTCTCGTGCGTGATTTCACTGCACGACGGCGTGGTCATGTATACGACCTCATCATTGACTACCACGCTCGGGTTTCCCAAGGACATGTGGATCGGCAGGTCGTTTATAGACTTCGTTCATCCGAGG
1719667 1719742
Getting Gene
>Troides_aeacus_Exon_7_CM067268.1_1719667_1719742_left_0_right_1
GATCGCAAAGCGTTCGCGTCGCAGATAACTAACGGTCTGGCCTTACCTAAGAATCTAAATGGTACGCAAAAAGAag
1718601 1718804
Getting Gene
>Troides_aeacus_Exon_8_CM067268.1_1718601_1718804_left_2_right_1
TTCAACCCCTGGAGAATTCAGTGTCTCCGATGGTGTGCAGAATAAGGCGCTACAAGGACCTATCTTCAGGTTTCGCCATAAAGAACAGCGTGATTACATACATGCCTTTCCTACTGAAATTATCCTTCAAGAACGTTAACGGTGAAGAGGGCAACGTCACCTACCTCGTTATACAAGCTGCTCCGTTTTCCTCGGCTTACAAAA
1717826 1718054
Getting Gene
>Troides_aeacus_Exon_9_CM067268.1_1717826_1718054_left_2_right_2
TATCGAACGAGAGCATCTCGAAGCCGGTGCCATTCGTCATCAGACACGCGGCAAACGGGAACTTGGAGTACCTAGACCCGGAATCGGTGCCGTACCTTGGCTACCTGCCGCAGGACGTAACCGAAATGGACGCGCTGCACCTCTACCACCCGTGCGACCTGTTTTACCTGAGACAGGTATATGAGATTATCGTCAAAGAAGGCGGATCG

>Troides_oblongomaculatus_Exon_16_JARDWC010000434.1_1760026_1760203_left_1_right_0
ACATAACGTGGAAATGGAGAAAGAGCTAATGAAGATGCATAGAGAGAGCCGCACCAGCAGCAAAGGTGAGAAAGAGAAGAACTCGAATGAAAAGAGACGGAAAAAAAAGGAACATTTGGCGCGATGTAACGCATCATTCCAACCGACATCTGCTAGAATGAACTTCACCGAAATAGCG
1759532 1759667
Getting Gene
>Troides_oblongomaculatus_Exon_17_JARDWC010000434.1_1759532_1759667_left_0_right_1
TCGCAGGGTGTGAAGAGGAATTCAAAGCAGCTTGAGTTCGAGGCCGGTGTCCCCAAGCATCACTGCACGTCCTCGCGGCAGTCCCGCCGCAGGCTAACGAACCCCGCAACAATGCACATTACTCCAACTGTCACCA
1759271 1759378
Getting Gene
>Troides_oblongomaculatus_Exon_18_JARDWC010000434.1_1759271_1759378_left_2_right_1
CTCCAACATCGTGGACGTCAAATTCAGTGAACAATATGAACGGTTTCATTTTTGGAGTTGGACTACCACAACAGATGTCCTTAATGAGCCCCATGGCGACCATGCCAG
1758853 1758966
Getting Gene
>Troides_oblongomaculatus_Exon_19_JARDWC010000434.1_1758853_1758966_left_2_right_1
GTTTATTCCCCATGCATTACGTACCCACGCCAGCGCCGCAAACATCTACAAACGCGACCGCGGGTAACACGTTACCAGGGGGCAGCTCGGCCGGCCAGTACCAGACGCCTCCGA
1758133 1758305
Getting Gene
>Troides_obl